# Deconvolution 1D

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

The illustration of 1D deconvolution or transpose convolution is explained in the figure below.While convolution preserves the length of input, deconv can be used to decompress a signal, where the output layer can have more neurons than inputs. Though the idea of deconv is difrerent from conv (or reverse meaning), we basically can deconv an input signal by using conv operations. To achieve it, we need to **pad input signal (y) with zeros and apply conv on it**.
<img src="./images/deconv_img1.jpeg" width="500">

### new_input_size =2(kernel_size -1) + input_size (because pytorch automatically adds 0 to both size of inputs)
### output_size = #new_input_size - (kernel_size - 1) = kernel_size + input_size -1

In [2]:
deconv = nn.ConvTranspose1d(in_channels=1, out_channels=1, kernel_size =3, stride = 1, padding = 0, bias = 0)
deconv.weight.data = torch.tensor([[[1,1,1]]], dtype = torch.float)

input = torch.tensor([[[1,1]]], dtype = torch.float)
output = deconv(input)

print("input =", input)
print("weight.data = ",deconv.weight.data  )
print("output =", output)


input = tensor([[[1., 1.]]])
weight.data =  tensor([[[1., 1., 1.]]])
output = tensor([[[1., 2., 2., 1.]]], grad_fn=<SqueezeBackward1>)


## Stride: 

When convolution, stride reduces the dimension of output signal by skipping some rows/cols. Thus, it makes sense that 
when we deconvoluate, the output signal increases. Remember, through deconv we dont want to resconstruct exact same signal (input of conv), but we accept some errors. This is because both weights (kernel weights) for conv and deconv are the best estimated. We can think of conv is finding set of weights $W$ so that $AW =B$, where A is a vector $nxn$ pixels (patch), W is weight vector of $[1xn^2]$. Thus, it can be easy to see that W is not invertible, meaning that finding the exact $W^-1$ is infesiable.  
The solution is estimate it and learn it through training. Anyway, the goal is we want to decompress the signal and it's weight can be learned in training. Pytorch do it by insert the number 0 between elements in input signals. 
Given example above, with stride = 2, input signal become [0,0,1,0,1,0,0] (includes padding).

Example below is kernel_size = 8, stride = 4( meaning, add three 0s betweens input units). 

### new_input_size = 2(kernel_size -1) + input_size + stride - 1 = 
### output_size = #new_input_size - (kernel_size - 1) =  input_size + kernel_size + stride - 2

In [3]:
deconv = nn.ConvTranspose1d(in_channels=1, out_channels=1, kernel_size = 8, stride = 4, padding = 0, bias = False)
deconv.weight.data = torch.ones(1,1,8)

input = torch.tensor([[[1,1]]], dtype = torch.float)
output = deconv(input)

print("input =", input)
print("weight.data = ",deconv.weight.data  )
print("output =", output)

input = tensor([[[1., 1.]]])
weight.data =  tensor([[[1., 1., 1., 1., 1., 1., 1., 1.]]])
output = tensor([[[1., 1., 1., 1., 2., 2., 2., 2., 1., 1., 1., 1.]]],
       grad_fn=<SqueezeBackward1>)


Observed that the length of output is longer. 
Size of output = #input + #stride + #kernel_size - 2

## Padding
Add #kernel_size -1 - padding number of 0s at both size of inputs.
### new_input_size = 2(kernel_size -1 - padding) + input_size + stride - 1 = 
### output_size = #new_input_size - (kernel_size - 1) =  input_size + kernel_size + stride - 2padding - 2

In [4]:
deconv = nn.ConvTranspose1d(in_channels=1, out_channels=1, kernel_size = 8, stride = 4, padding = 2, bias = False)
deconv.weight.data = torch.ones(1,1,8)

input = torch.tensor([[[1,1]]], dtype = torch.float)
output = deconv(input)

print("input =", input)
print("weight.data = ",deconv.weight.data  )
print("output =", output)


input = tensor([[[1., 1.]]])
weight.data =  tensor([[[1., 1., 1., 1., 1., 1., 1., 1.]]])
output = tensor([[[1., 1., 2., 2., 2., 2., 1., 1.]]], grad_fn=<SqueezeBackward1>)


# output_padding

Pytorch allows pads zeros ($<$ stride) to output to help achive the right output dimension.


In [5]:
deconv = nn.ConvTranspose1d(in_channels=1, out_channels=1, kernel_size = 8, stride = 4, output_padding = 3, bias = False)
deconv.weight.data = torch.ones(1,1,8)

input = torch.tensor([[[1,1]]], dtype = torch.float)
output = deconv(input)

print("input =", input)
print("weight.data = ",deconv.weight.data  )
print("output =", output)


input = tensor([[[1., 1.]]])
weight.data =  tensor([[[1., 1., 1., 1., 1., 1., 1., 1.]]])
output = tensor([[[1., 1., 1., 1., 2., 2., 2., 2., 1., 1., 1., 1., 0., 0., 0.]]],
       grad_fn=<SqueezeBackward1>)


## Output_padding + padding

This combines 2 scenarios above. First, it adds padding 0's at both sizes of inputs, deconv input the results is
output = tensor([[[1., 1., 2., 2., 2., 2., 1., 1.]]]. 
Next, it applies output_padding, where it adds #output_padding at the end of output. **Note that, it does not add 0's but instead it's add 1, because this is what removed from padding in previous step. If #output_padding > #padding, 0 will be add. **


In [6]:
deconv = nn.ConvTranspose1d(in_channels=1, out_channels=1, kernel_size = 8, stride = 4, padding = 2, output_padding = 3, bias = False)
deconv.weight.data = torch.ones(1,1,8)

input = torch.tensor([[[1,1]]], dtype = torch.float)
output = deconv(input)

print("input =", input)
print("weight.data = ",deconv.weight.data  )
print("output =", output)


input = tensor([[[1., 1.]]])
weight.data =  tensor([[[1., 1., 1., 1., 1., 1., 1., 1.]]])
output = tensor([[[1., 1., 2., 2., 2., 2., 1., 1., 1., 1., 0.]]],
       grad_fn=<SqueezeBackward1>)


## References

https://medium.com/apache-mxnet/transposed-convolutions-explained-with-ms-excel-52d13030c7e8  
https://distill.pub/2016/deconv-checkerboard/  
https://towardsdatascience.com/is-the-transposed-convolution-layer-and-convolution-layer-the-same-thing-8655b751c3a1  
https://medium.com/@santi.pdp/how-pytorch-transposed-convs1d-work-a7adac63c4a5  


